In [1]:
negativeBagOfWords = []
positiveBagOfWords = []
corpusReview = []
corpusSentiment = []
positiveSentiments = []
negativeSentiments = []


In [2]:

stopWords = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]


In [11]:

def stemming(review):
	listOfTokens = []
	word = ""
	flag = 0
	for i in range(len(review)):

		if review[i].isalpha():
			if i==len(review)-1:
				word += review[i]
				listOfTokens.append(word)
				break
			elif flag==0:
				word += review[i]
				flag = 1
			else:
				word += review[i]
		else:
			if flag==1:
				# Lowering the case
				word = word.lower()

				# Stopword check Implementaion
				if word not in stopWords:
					listOfTokens.append(word)
				flag=0
				word = ""
			else:
				continue
	return	listOfTokens


In [12]:
def createBagOfWords():
	global negativeSentiments, positiveSentiments
	pos = {}
	neg = {}
	cnt = int(-1)
	index = int(0)
	with open('a1_d3.txt', 'r') as filehandle:
		for line in filehandle:
			cnt += 1
			line  = line.split('	')
			review = line[0][:-1]
			sentiment = int(line[1])
			review = stemming(review)
			corpusReview[index].append(review)
			corpusSentiment[index].append(sentiment)
			if sentiment == int(1):
				positiveSentiments[index] += 1
				for token in review:
					if token in positiveBagOfWords[index].keys():
						positiveBagOfWords[index][token] += 1
					else:
						positiveBagOfWords[index][token] = 1
			else:
				negativeSentiments[index] += 1
				for token in review:
					if token in negativeBagOfWords[index].keys():
						negativeBagOfWords[index][token] += 1
					else:
						negativeBagOfWords[index][token] = 1

			if cnt==int(199):
				index += 1
				cnt = int(-1)
				neg.clear()
				pos.clear()


In [13]:
def cvHelper(index,k):
	testX = corpusReview[index]
	testY = corpusSentiment[index]
	TP = int(0)
	TN = int(0)
	FP = int(0)
	FN = int(0)
	cnt = -1
	for review in testX:
		cnt += 1
		likelihood_1 = 1
		likelihood_0 = 1
		for token in review:
			flagP = 0
			flagN = 0
			P = 0
			N = 0
			for i in range(k):
				if i==index:
					continue
				dicP = positiveBagOfWords[i]
				dicN = negativeBagOfWords[i]
				if token in dicP.keys():
					P += dicP[token]
					flagP += 1
				if token in dicN.keys():
					N += dicN[token]
					flagN += 1
			if flagP==0 and flagN==0:
				continue
			else:
				# Laplace Smoothing
				likelihood_1 *= (P + 1)
				likelihood_0 *= (N  + 1)

		# Lallace Smoothing
		posterior_1_est = (likelihood_1) * (positiveSentiments[index]  +  1)

		posterior_0_est = (likelihood_0) * (negativeSentiments[index]  + 1)

		calY = 0
		if posterior_1_est > posterior_0_est:
			calY += 1
		if testY[cnt]==1 and calY==1:
			TP += 1
		elif testY[cnt]==0 and calY==0:
			TN += 1
		elif testY[cnt]==0 and calY==1:
			FP += 1
		elif testY[cnt]==1 and calY==0:
			FN += 1
	accuracy = (TP+TN)/(TP + TN + FP + FN)
	recall = (TP) / (TP + FN)
	precision = (TP)/(TP + FP)
	fScore = (2*recall*precision)/(recall + precision)
	return accuracy,fScore


In [14]:
def kFoldCrossValidation(k):
	accList = []
	fScoreList = []
	for index in range(k):
		accuracy,fScore =  cvHelper(index,k)
		accList.append(accuracy)
		fScoreList.append(fScore)

	return accList,fScoreList

def calMeanVariance(A):
	mu = (sum(A))/(len(A))
	sumOfSq = 0
	for val in A:
		sumOfSq += (val-mu) ** 2
	var = sumOfSq/len(A)
	return mu,var


In [15]:
for i in range(5):
	positiveBagOfWords.append({})
	negativeBagOfWords.append({})
	corpusReview.append([])
	corpusSentiment.append([])
	positiveSentiments.append(int(0))
	negativeSentiments.append(int(0))

createBagOfWords()
A = kFoldCrossValidation(5)
Accuracy = calMeanVariance(A[0])
fScore = calMeanVariance(A[1])
print(A)
print("Accuracy : ",Accuracy[0],u"\u00B1",Accuracy[1])
print("F Score : ",fScore[0],u"\u00B1",fScore[1])

([0.8225, 0.8275, 0.76, 0.7875, 0.76], [0.8313539192399049, 0.8470066518847006, 0.7767441860465116, 0.7745358090185676, 0.7303370786516854])
Accuracy :  0.7915000000000001 ± 0.0008515000000000002
F Score :  0.791995528968274 ± 0.001782903261276559
